In [ ]:
#yes bye finding nearest node
import torch
import torch.nn as nn
import torch.optim as optim
import random

# MAC Protocol Constants
PROTOCOLS = ["Protocol1", "Protocol2", "Protocol3", "Protocol4", "UGAV-MDVR"]

class CommunicationModel(nn.Module):
    def __init__(self):
        super(CommunicationModel, self).__init__()
        # neural network model
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

class GPS:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

class SignalStrengthModel(nn.Module):
    def __init__(self):
        super(SignalStrengthModel, self).__init__()
        # neural model for signal strength prediction
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

class ClusterCoordinatingSystem:
    def __init__(self):
        # Initialize cluster-based coordinating system attributes
        # Add necessary attributes for the (UGAV-MDVR) protocol.
        self.signal_strength_model = SignalStrengthModel()

    def level_based_cluster_head_election(self):
        # Implement the LBCHE scheme here.
        pass

    def disseminate_mdm(self, source, mdm, relay_to=None):
        # Implementing MDM dissemination logic here.
        if relay_to is not None:
            print(f"Relaying message from Robot {source.robot_id} to Robot {relay_to.robot_id}: {mdm}")
            relay_to.receive_message(mdm)
            relay_to.energy -= 1  #we can adjust

    def calculate_signal_strength(self, sender, receiver):
        # Calculating signal strength 
        distance = ((receiver.gps.latitude - sender.gps.latitude) ** 2 + (receiver.gps.longitude - sender.gps.longitude) ** 2) ** 0.5
        input_data = torch.tensor([sender.gps.latitude, sender.gps.longitude, receiver.gps.latitude, receiver.gps.longitude], dtype=torch.float32).unsqueeze(0)
        signal_strength = self.signal_strength_model(input_data).item()
        return signal_strength

class Robot:
    def __init__(self, robot_id, latitude, longitude, energy=100, mac_protocol=None, vehicle_type="UGV"):
        self.robot_id = robot_id
        self.gps = GPS(latitude, longitude)
        self.energy = energy
        self.mac_protocol = mac_protocol
        self.vehicle_type = vehicle_type
        self.messages_received = []

    def send_message(self, other_robot, message, coordinating_system):
        # incorporating the (UGAV-MDVR) protocol and MDM dissemination.
        signal_strength = coordinating_system.calculate_signal_strength(self, other_robot)

        print(f"Robot {self.robot_id} is sending a message to Robot {other_robot.robot_id}: {message}")
        print(f"Signal Strength: {signal_strength}")

        # Receiving the message if the signal strength is above a certain threshold
        if signal_strength > 0.1:
            coordinating_system.disseminate_mdm(self, message, relay_to=None)
            self.energy -= 2  # we can adjust
        else:
            # Finding a nearby robot for sending message
            nearby_robot = self.find_nearby_robot(coordinating_system)
            if nearby_robot is not None:
                print(f"Robot {self.robot_id} is relaying the message through Robot {nearby_robot.robot_id}")
                coordinating_system.disseminate_mdm(self, message, relay_to=nearby_robot)
            else:
                print("Message not relayed due to lack of nearby robots.")

        other_robot.receive_message(message) 
        other_robot.energy -= 1  # Adjust

    def receive_message(self, message):
        print(f"Robot {self.robot_id} received a message: {message}")
        self.messages_received.append(message)
        self.energy -= 1  # Adjust 
    def find_nearby_robot(self, coordinating_system):
        for robot in ranet:
            if robot.robot_id != self.robot_id:
                distance = ((robot.gps.latitude - self.gps.latitude) ** 2 + (robot.gps.longitude - self.gps.longitude) ** 2) ** 0.5
                if distance < 10:  # Adjust
                    return robot
        return None

def main():
    ranet = []
    coordinating_system = ClusterCoordinatingSystem()

    for i in range(5):
        ranet.append(Robot(i + 1, random.uniform(0, 90), random.uniform(0, 180), mac_protocol=PROTOCOLS[i]))

    communication_model = CommunicationModel()
    optimizer = optim.Adam(communication_model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()

    while True:
        sender_id = int(input("Enter the ID of the sender robot (or enter -1 to exit): "))
        if sender_id == -1:
            break

        receiver_id = int(input("Enter the ID of the receiver robot: "))

        sender = next((r for r in ranet if r.robot_id == sender_id), None)
        receiver = next((r for r in ranet if r.robot_id == receiver_id), None)

        # Prompt for message input
        message = input(f"Enter the message from Robot {sender_id} to Robot {receiver_id}: ")

        if sender is not None and receiver is not None:
            # Simulate communication
            input_data = torch.tensor([sender.gps.latitude, sender.gps.longitude, receiver.gps.latitude, receiver.gps.longitude], dtype=torch.float32).unsqueeze(0)
            target = torch.tensor([1.0 if random.random() < 0.5 else 0.0], dtype=torch.float32).unsqueeze(0)

            optimizer.zero_grad()
            output = communication_model(input_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            # Sending a message based on the model's prediction
            if output.item() > 0.5:
                sender.send_message(receiver, message, coordinating_system)

        else:
            print("Invalid sender or receiver ID. Please try again.")

    # Displaying final info
    for robot in ranet:
        print(f"Robot {robot.robot_id}: GPS=({robot.gps.latitude}, {robot.gps.longitude}), Energy={robot.energy}, MAC Protocol={robot.mac_protocol}")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    main()

In [ ]:
#yes with signal strength
import torch
import torch.nn as nn
import torch.optim as optim
import random

# MAC Protocol Constants
PROTOCOLS = ["Protocol1", "Protocol2", "Protocol3", "Protocol4", "UGAV-MDVR"]

class CommunicationModel(nn.Module):
    def __init__(self):
        super(CommunicationModel, self).__init__()
        # neural network model
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

class GPS:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

class SignalStrengthModel(nn.Module):
    def __init__(self):
        super(SignalStrengthModel, self).__init__()
        # neural network model for signal strength prediction
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

class ClusterCoordinatingSystem:
    def __init__(self):
        # Initialize cluster-based coordinating system attributes
        # Add necessary attributes for the (UGAV-MDVR) protocol.
        self.signal_strength_model = SignalStrengthModel()

    def level_based_cluster_head_election(self):
        # Implement the LBCHE scheme here.
        pass

    def disseminate_mdm(self, source, mdm):
        # Implement MDM dissemination logic here.
        pass

    def calculate_signal_strength(self, sender, receiver):
        # Calculate signal strength based on distance
        distance = ((receiver.gps.latitude - sender.gps.latitude) ** 2 + (receiver.gps.longitude - sender.gps.longitude) ** 2) ** 0.5
        input_data = torch.tensor([sender.gps.latitude, sender.gps.longitude, receiver.gps.latitude, receiver.gps.longitude], dtype=torch.float32).unsqueeze(0)
        signal_strength = self.signal_strength_model(input_data).item()
        return signal_strength

class Robot:
    def __init__(self, robot_id, latitude, longitude, energy=100, mac_protocol=None, vehicle_type="UGV"):
        self.robot_id = robot_id
        self.gps = GPS(latitude, longitude)
        self.energy = energy
        self.mac_protocol = mac_protocol
        self.vehicle_type = vehicle_type
        self.messages_received = []

    def send_message(self, other_robot, message, coordinating_system):
        # Modify this method to incorporate the (UGAV-MDVR) protocol and MDM dissemination.
        signal_strength = coordinating_system.calculate_signal_strength(self, other_robot)

        print(f"Robot {self.robot_id} is sending a message to Robot {other_robot.robot_id}: {message}")
        print(f"Signal Strength: {signal_strength}")

        # Receive the message if the signal strength is above a certain threshold
        if signal_strength > 0.1:
            coordinating_system.disseminate_mdm(self, message)
            self.energy -= 2  # Adjust energy consumption for the UGAV-MDVR protocol
        else:
            print("Message not received due to weak signal.")

        other_robot.receive_message(message)  # Move the message reception here
        other_robot.energy -= 1  # Adjust energy consumption for the receiving robot

    def receive_message(self, message):
        print(f"Robot {self.robot_id} received a message: {message}")
        self.messages_received.append(message)
        self.energy -= 1  # Adjust energy consumption for receiving a message

def main():
    ranet = []
    coordinating_system = ClusterCoordinatingSystem()

    for i in range(5):
        ranet.append(Robot(i + 1, random.uniform(0, 90), random.uniform(0, 180), mac_protocol=PROTOCOLS[i]))

    communication_model = CommunicationModel()
    optimizer = optim.Adam(communication_model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()

    while True:
        sender_id = int(input("Enter the ID of the sender robot (or enter -1 to exit): "))
        if sender_id == -1:
            break

        receiver_id = int(input("Enter the ID of the receiver robot: "))

        sender = next((r for r in ranet if r.robot_id == sender_id), None)
        receiver = next((r for r in ranet if r.robot_id == receiver_id), None)

        # Prompt for message input
        message = input(f"Enter the message from Robot {sender_id} to Robot {receiver_id}: ")

        if sender is not None and receiver is not None:
            # Simulate communication
            input_data = torch.tensor([sender.gps.latitude, sender.gps.longitude, receiver.gps.latitude, receiver.gps.longitude], dtype=torch.float32).unsqueeze(0)
            target = torch.tensor([1.0 if random.random() < 0.5 else 0.0], dtype=torch.float32).unsqueeze(0)

            optimizer.zero_grad()
            output = communication_model(input_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            # Send a message based on the model's prediction
            if output.item() > 0.5:
                sender.send_message(receiver, message, coordinating_system)

        else:
            print("Invalid sender or receiver ID. Please try again.")

    # Displaying final info
    for robot in ranet:
        print(f"Robot {robot.robot_id}: GPS=({robot.gps.latitude}, {robot.gps.longitude}), Energy={robot.energy}, MAC Protocol={robot.mac_protocol}")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    main()

In [ ]:
#works
import torch
import torch.nn as nn
import torch.optim as optim
import random

# MAC Protocol Constants
PROTOCOLS = ["Protocol1", "Protocol2", "Protocol3", "Protocol4", "UGAV-MDVR"]

class CommunicationModel(nn.Module):
    def __init__(self):
        super(CommunicationModel, self).__init__()
        # neural network model
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

class GPS:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

class ClusterCoordinatingSystem:
    def __init__(self):
        # Initialize cluster-based coordinating system attributes
        # Add necessary attributes for the (UGAV-MDVR) protocol.
        pass

    def level_based_cluster_head_election(self):
        # Implement the LBCHE scheme here.
        pass

    def disseminate_mdm(self, source, mdm):
        # Implement MDM dissemination logic here.
        pass

class Robot:
    def __init__(self, robot_id, latitude, longitude, energy=100, mac_protocol=None, vehicle_type="UGV"):
        self.robot_id = robot_id
        self.gps = GPS(latitude, longitude)
        self.energy = energy
        self.mac_protocol = mac_protocol
        self.vehicle_type = vehicle_type
        self.messages_received = []

    def send_message(self, other_robot, message, coordinating_system):
        # Modify this method to incorporate the (UGAV-MDVR) protocol and MDM dissemination.
        if self.mac_protocol == "UGAV-MDVR":
            coordinating_system.disseminate_mdm(self, message)
            self.energy -= 2  # Adjust energy consumption for the UGAV-MDVR protocol
        else:
            # Adjust energy consumption for other protocols if needed.
            self.energy -= 1  # Adjust the energy consumption for other protocols

        other_robot.receive_message(message)  # Move the message reception here
        other_robot.energy -= 1  # Adjust energy consumption for the receiving robot

    def receive_message(self, message):
        print(f"Robot {self.robot_id} received a message: {message}")
        self.messages_received.append(message)
        self.energy -= 1  # Adjust energy consumption for receiving a message


def main():
    ranet = []
    coordinating_system = ClusterCoordinatingSystem()

    for i in range(5):
        ranet.append(Robot(i + 1, random.uniform(0, 90), random.uniform(0, 180), mac_protocol=PROTOCOLS[i]))

    communication_model = CommunicationModel()
    optimizer = optim.Adam(communication_model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()

    while True:
        sender_id = int(input("Enter the ID of the sender robot (or enter -1 to exit): "))
        if sender_id == -1:
            break

        receiver_id = int(input("Enter the ID of the receiver robot: "))

        sender = next((r for r in ranet if r.robot_id == sender_id), None)
        receiver = next((r for r in ranet if r.robot_id == receiver_id), None)

        # Prompt for message input
        message = input(f"Enter the message from Robot {sender_id} to Robot {receiver_id}: ")

        if sender is not None and receiver is not None:
            # Simulate communication
            input_data = torch.tensor([sender.gps.latitude, sender.gps.longitude, receiver.gps.latitude, receiver.gps.longitude], dtype=torch.float32).unsqueeze(0)
            target = torch.tensor([1.0 if random.random() < 0.5 else 0.0], dtype=torch.float32).unsqueeze(0)

            optimizer.zero_grad()
            output = communication_model(input_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            # Send a message based on the model's prediction
            if output.item() > 0.5:
                sender.send_message(receiver, message, coordinating_system)

        else:
            print("Invalid sender or receiver ID. Please try again.")

    # Displaying final info
    for robot in ranet:
        print(f"Robot {robot.robot_id}: GPS=({robot.gps.latitude}, {robot.gps.longitude}), Energy={robot.energy}, MAC Protocol={robot.mac_protocol}")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    main()


In [ ]:
#arp
#ARP YES
import torch
import torch.nn as nn
import torch.optim as optim
import random

# MAC Protocol Constants
PROTOCOLS = ["Protocol1", "Protocol2", "Protocol3", "Protocol4", "Protocol5"]

ranet = []  # Declare ranet at the module level

class Robot:
    def __init__(self, robot_id, x, y, energy=100, mac_protocol=None, ip_address=None):
        self.robot_id = robot_id
        self.x = x
        self.y = y
        self.energy = energy
        self.mac_protocol = mac_protocol
        self.ip_address = ip_address
        self.messages_received = []

    def move(self, new_x, new_y):
        distance = ((new_x - self.x) ** 2 + (new_y - self.y) ** 2) ** 0.5
        energy_consumed = distance * 2
        if self.energy >= energy_consumed:
            print(f"Robot {self.robot_id} is moving from ({self.x}, {self.y}) to ({new_x}, {new_y})")
            self.x = new_x
            self.y = new_y
            self.energy -= energy_consumed
            print(f"Energy consumed: {energy_consumed}, Remaining energy: {self.energy}")
        else:
            print(f"Robot {self.robot_id} does not have enough energy to move.")

    def calculate_signal_strength(self, other_robot):
        distance = ((other_robot.x - self.x) ** 2 + (other_robot.y - self.y) ** 2) ** 0.5
        attenuation_factor = 1.0 / (distance + 1.0)
        return attenuation_factor

    def send_message(self, other_robot, message, communication_model):
        destination_id = communication_model.arp_resolution(other_robot.ip_address)

        print(f"Robot {self.robot_id} is sending a message to Robot {destination_id}: {message}")
        signal_strength = self.calculate_signal_strength(other_robot)

        if signal_strength > 0.1:
            other_robot.receive_message(message)
        else:
            print("Message not received due to weak signal.")

        # Move energy deduction outside the condition
        self.energy -= 2
        print(f"Signal Strength: {signal_strength}")

    def receive_message(self, message):
        print(f"Robot {self.robot_id} received a message: {message}")
        self.messages_received.append(message)

class CommunicationModel(nn.Module):
    def __init__(self):
        super(CommunicationModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

    def arp_resolution(self, ip_address):
        for robot in ranet:
            if robot.ip_address == ip_address:
                return robot.robot_id
        return None

def main():
    global ranet  # Use the global ranet variable

    for i in range(5):
        ip_address = f"192.168.1.{i+1}"
        ranet.append(Robot(i + 1, random.randint(0, 10), random.randint(0, 10), mac_protocol=PROTOCOLS[i], ip_address=ip_address))

    communication_model = CommunicationModel()
    optimizer = optim.Adam(communication_model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()

    while True:
        sender_id = int(input("Enter the ID of the sender robot (or enter -1 to exit): "))
        if sender_id == -1:
            break

        receiver_id = int(input("Enter the ID of the receiver robot: "))

        sender = next((r for r in ranet if r.robot_id == sender_id), None)
        receiver = next((r for r in ranet if r.robot_id == receiver_id), None)

        message = input(f"Enter the message from Robot {sender_id} to Robot {receiver_id}: ")

        if sender is not None and receiver is not None:
            # Pass communication_model as an argument
            sender.send_message(receiver, message, communication_model)

        else:
            print("Invalid sender or receiver ID. Please try again.")

    for robot in ranet:
        print(f"Robot {robot.robot_id}: Position=({robot.x}, {robot.y}), Energy={robot.energy}, MAC Protocol={robot.mac_protocol}, IP Address={robot.ip_address}")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    communication_model = CommunicationModel()  # Move this line outside of the main function
    main()


In [ ]:
#basic
import simpy
import random

class Robot:
    def __init__(self, env, name, initial_energy=100):
        self.env = env
        self.name = name
        self.channel_busy = False
        self.energy = initial_energy

    def distance_to(self, other_robot):
        return abs(ord(self.name[-1]) - ord(other_robot.name[-1]))

    def calculate_signal_strength(self, other_robot):
        distance = self.distance_to(other_robot)
        max_distance = 10
        normalized_distance = min(distance / max_distance, 1.0)
        return 1.0 - normalized_distance

    def send_data(self, destination, message):
        with destination.channel.request() as req:
            yield req

            if not destination.channel_busy:
                if self.energy > 0:
                    signal_strength = self.calculate_signal_strength(destination)
                    print(f"{self.name} is sending message '{message}' to {destination.name} with signal strength {signal_strength:.2f}")
                    self.energy -= 1
                    yield self.env.timeout(1)  # Simulating data transmission time
                    print(f"{self.name}'s message successfully sent to {destination.name}")
                else:
                    print(f"{self.name} does not have enough energy to transmit.")
            else:
                print(f"Channel busy. {self.name}'s message transmission to {destination.name} failed.")

def csma_ca(env, robots):
    while True:
        yield env.timeout(random.uniform(0, 5))  # Random backoff time

        sender_robot = None
        while not sender_robot:
            sender_name = input("Enter the name of the sender robot: ")
            sender_robot = next((robot for robot in robots if robot.name == sender_name), None)
            if not sender_robot:
                print("Invalid sender name. Try again.")

        receiver_robot = None
        while not receiver_robot or receiver_robot == sender_robot:
            receiver_name = input("Enter the name of the receiver robot: ")
            receiver_robot = next((robot for robot in robots if robot.name == receiver_name), None)
            if not receiver_robot:
                print("Invalid receiver name. Try again.")
            elif receiver_robot == sender_robot:
                print("Receiver cannot be the same as the sender. Try again.")

        message = input(f"Enter the message for {sender_robot.name}: ")

        yield env.process(sender_robot.send_data(receiver_robot, message))

# Get user inputs
num_robots = int(input("Enter the number of robots: "))
simulation_duration = int(input("Enter the simulation duration (in time units): "))

robots = []
env = simpy.Environment()

for i in range(1, num_robots + 1):
    initial_energy = int(input(f"Enter initial energy for Robot{i}: "))
    robot = Robot(env, f"Robot{i}", initial_energy)
    robot.channel = simpy.Resource(env)
    robots.append(robot)

env.process(csma_ca(env, robots))

env.run(until=simulation_duration)

# Print energy consumption of each robot after the simulation
for robot in robots:
    print(f"{robot.name}: Energy = {robot.energy}")


In [ ]:
import simpy
import random

class Robot:
    def __init__(self, env, name, initial_energy=100, signal_strength=1.0):
        self.env = env
        self.name = name
        self.channel_busy = False
        self.energy = initial_energy
        self.signal_strength = signal_strength

    def send_data(self, destination):
        with destination.channel.request() as req:
            yield req

            if not destination.channel_busy:
                if self.energy > 0 and self.signal_strength > 0:
                    print(f"{self.name} is sending data to {destination.name}")
                    self.energy -= 1
                    yield self.env.timeout(1)  # Simulating data transmission time
                    print(f"{self.name}'s data successfully sent to {destination.name}")
                else:
                    print(f"{self.name} does not have enough energy or signal strength to transmit.")
            else:
                print(f"Channel busy. {self.name}'s data transmission to {destination.name} failed.")

def csma_ca(env, robots):
    while True:
        yield env.timeout(random.uniform(0, 5))  # Random backoff time

        resource_requests = [robot.channel.request() for robot in robots]
        results = yield simpy.AnyOf(env, resource_requests)  # Wait for any of the robots to be ready

        transmitting_robot = None
        for i, result in enumerate(results):
            if result.ok:
                transmitting_robot = robots[i]
                break

        if transmitting_robot and not any(robot.channel_busy for robot in robots):
            transmitting_robot.channel_busy = True
            yield env.timeout(1)  # Channel is busy for data transmission
            transmitting_robot.channel_busy = False
        else:
            print("Collision detected. Retrying...")

# Get user inputs
num_robots = int(input("Enter the number of robots: "))
simulation_duration = int(input("Enter the simulation duration (in time units): "))

robots = []
env = simpy.Environment()

for i in range(1, num_robots + 1):
    initial_energy = int(input(f"Enter initial energy for Robot{i}: "))
    signal_strength = float(input(f"Enter signal strength for Robot{i} (between 0 and 1): "))
    robot = Robot(env, f"Robot{i}", initial_energy, signal_strength)
    robot.channel = simpy.Resource(env)
    robots.append(robot)

env.process(csma_ca(env, robots))

for i in range(len(robots)):
    for j in range(i+1, len(robots)):
        env.process(robots[i].send_data(robots[j]))

env.run(until=simulation_duration)

# Print energy consumption and signal strength of each robot after the simulation
for robot in robots:
    print(f"{robot.name}: Energy = {robot.energy}, Signal Strength = {robot.signal_strength}")


In [ ]:
import simpy
import random

class Robot:
    def __init__(self, env, name, initial_energy=100, signal_strength=1.0):
        self.env = env
        self.name = name
        self.channel_busy = False
        self.energy = initial_energy
        self.signal_strength = signal_strength

    def send_data(self, destination):
        with destination.channel.request() as req:
            yield req

            if not destination.channel_busy:
                if self.energy > 0 and self.signal_strength > 0:
                    print(f"{self.name} is sending data to {destination.name}")
                    self.energy -= 1
                    yield self.env.timeout(1)  # Simulating data transmission time
                    print(f"{self.name}'s data successfully sent to {destination.name}")
                else:
                    print(f"{self.name} does not have enough energy or signal strength to transmit.")
            else:
                print(f"Channel busy. {self.name}'s data transmission to {destination.name} failed.")

def csma_ca(env, robots):
    while True:
        yield env.timeout(random.uniform(0, 5))  # Random backoff time

        resource_requests = [robot.channel.request() for robot in robots]
        results = yield simpy.AnyOf(env, resource_requests)  # Wait for any of the robots to be ready

        transmitting_robot = None
        for i, result in enumerate(results):
            if result.ok:
                transmitting_robot = robots[i]
                break

        if transmitting_robot and not any(robot.channel_busy for robot in robots):
            transmitting_robot.channel_busy = True
            yield env.timeout(1)  # Channel is busy for data transmission
            transmitting_robot.channel_busy = False
        else:
            print("Collision detected. Retrying...")


env = simpy.Environment()

robots = [
    Robot(env, "Robot1", initial_energy=50, signal_strength=0.8),
    Robot(env, "Robot2", initial_energy=60, signal_strength=1.0),
    Robot(env, "Robot3", initial_energy=70, signal_strength=0.9),
    Robot(env, "Robot4", initial_energy=80, signal_strength=0.7),
    Robot(env, "Robot5", initial_energy=90, signal_strength=0.95),
]

for robot in robots:
    robot.channel = simpy.Resource(env)

env.process(csma_ca(env, robots))

for i in range(len(robots)):
    for j in range(i+1, len(robots)):
        env.process(robots[i].send_data(robots[j]))

env.run(until=10)

# Print energy consumption and signal strength of each robot after the simulation
for robot in robots:
    print(f"{robot.name}: Energy = {robot.energy}, Signal Strength = {robot.signal_strength}")


In [ ]:
#mac
#yes
#strength of signal,MAC protocol,location of the robots
import torch
import torch.nn as nn
import torch.optim as optim
import random

# MAC Protocol Constants
PROTOCOLS = ["Protocol1", "Protocol2", "Protocol3", "Protocol4", "Protocol5"]

class Robot:
    def __init__(self, robot_id, x, y, energy=100, mac_protocol=None):
        self.robot_id = robot_id
        self.x = x
        self.y = y
        self.energy = energy
        self.mac_protocol = mac_protocol
        self.messages_received = []

    def move(self, new_x, new_y):
        distance = ((new_x - self.x) ** 2 + (new_y - self.y) ** 2) ** 0.5
        energy_consumed = distance * 2
        if self.energy >= energy_consumed:
            print(f"Robot {self.robot_id} is moving from ({self.x}, {self.y}) to ({new_x}, {new_y})")
            self.x = new_x
            self.y = new_y
            self.energy -= energy_consumed
            print(f"Energy consumed: {energy_consumed}, Remaining energy: {self.energy}")
        else:
            print(f"Robot {self.robot_id} does not have enough energy to move.")

    def calculate_signal_strength(self, other_robot):
        #Signal strength based on distance
        distance = ((other_robot.x - self.x) ** 2 + (other_robot.y - self.y) ** 2) ** 0.5
        # Assuming a simple linear attenuation model
        attenuation_factor = 1.0 / (distance + 1.0)  #to avoid division by zero
        return attenuation_factor

    def send_message(self, other_robot, message):
        # Calculating signal strength
        signal_strength = self.calculate_signal_strength(other_robot)

        print(f"Robot {self.robot_id} is sending a message to Robot {other_robot.robot_id}: {message}")
        print(f"Signal Strength: {signal_strength}")
        
        # receive the message if the signal strength is above a certain threshold
        if signal_strength > 0.1:
            other_robot.receive_message(message)
            self.energy -= 2  #energy consumption(assuming)
        else:
            print("Message not received due to weak signal.")

    def receive_message(self, message):
        print(f"Robot {self.robot_id} received a message: {message}")
        self.messages_received.append(message)

class CommunicationModel(nn.Module):
    def __init__(self):
        super(CommunicationModel, self).__init__()
        # neural network model
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

def main():
    ranet = []

    for i in range(5):
        ranet.append(Robot(i + 1, random.randint(0, 10), random.randint(0, 10), mac_protocol=PROTOCOLS[i]))

    communication_model = CommunicationModel()
    optimizer = optim.Adam(communication_model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()

    while True:
        sender_id = int(input("Enter the ID of the sender robot (or enter -1 to exit): "))
        if sender_id == -1:
            break

        receiver_id = int(input("Enter the ID of the receiver robot: "))

        sender = next((r for r in ranet if r.robot_id == sender_id), None)
        receiver = next((r for r in ranet if r.robot_id == receiver_id), None)

        # Prompt for message input 
        message = input(f"Enter the message from Robot {sender_id} to Robot {receiver_id}: ")

        if sender is not None and receiver is not None:
            # Simulate communication
            input_data = torch.tensor([sender.x, sender.y, receiver.x, receiver.y], dtype=torch.float32).unsqueeze(0)
            target = torch.tensor([1.0 if random.random() < 0.5 else 0.0], dtype=torch.float32).unsqueeze(0)

            optimizer.zero_grad()
            output = communication_model(input_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            # Send a message based on the model's prediction
            if output.item() > 0.5:
                sender.send_message(receiver, message)

        else:
            print("Invalid sender or receiver ID. Please try again.")

    # Displaying final info
    for robot in ranet:
        print(f"Robot {robot.robot_id}: Position=({robot.x}, {robot.y}), Energy={robot.energy}, MAC Protocol={robot.mac_protocol}")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    main()

In [ ]:
#sending direct
def main():
    ranet = []

    # Creating robots with initial positions
    for i in range(5):
        ranet.append(Robot(i + 1, random.randint(0, 10), random.randint(0, 10)))

    communication_model = CommunicationModel()
    optimizer = optim.Adam(communication_model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()

    # Training the communication model
    train_communication = input("Do you want to train the communication model? (yes/no): ").lower() == 'yes'

    if train_communication:
        num_epochs = int(input("Enter the number of training epochs: "))
        for epoch in range(num_epochs):
            for robot in ranet:
                for other_robot in ranet:
                    if robot != other_robot:
                        # Simulate communication
                        input_data = torch.tensor([robot.x, robot.y, other_robot.x, other_robot.y], dtype=torch.float32).unsqueeze(0)
                        target = torch.tensor([1.0 if random.random() < 0.5 else 0.0], dtype=torch.float32).unsqueeze(0)

                        optimizer.zero_grad()
                        output = communication_model(input_data)
                        loss = criterion(output, target)
                        loss.backward()
                        optimizer.step()

                        # Send a message based on the model's prediction
                        if output.item() > 0.5:
                            robot.send_message(other_robot, f"Hello from Robot {robot.robot_id}!")

    #sender and receiver input from the user
    sender_id = int(input("Enter the ID of the sender robot: "))
    receiver_id = int(input("Enter the ID of the receiver robot: "))
    sender = next((robot for robot in ranet if robot.robot_id == sender_id), None)
    receiver = next((robot for robot in ranet if robot.robot_id == receiver_id), None)

    if sender is not None and receiver is not None:
        sender.send_message(receiver, f"Direct message from Robot {sender.robot_id} to Robot {receiver.robot_id}!")

    # Displaying final info
    for robot in ranet:
        print(f"Robot {robot.robot_id}: Position=({robot.x}, {robot.y})")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    main()


In [ ]:
#direct message transfer
#with added energy parameter
import torch
import torch.nn as nn
import torch.optim as optim
import random

class Robot:
    def __init__(self, robot_id, x, y, energy=100):
        self.robot_id = robot_id
        self.x = x
        self.y = y
        self.energy = energy
        self.messages_received = []

    def move(self, new_x, new_y):
        distance = ((new_x - self.x) ** 2 + (new_y - self.y) ** 2) ** 0.5
        energy_consumed = distance * 2
        if self.energy >= energy_consumed:
            print(f"Robot {self.robot_id} is moving from ({self.x}, {self.y}) to ({new_x}, {new_y})")
            self.x = new_x
            self.y = new_y
            self.energy -= energy_consumed
        else:
            print(f"Robot {self.robot_id} does not have enough energy to move.")

    def send_message(self, other_robot, message):
        if self.energy >= 10:  # Assuming 10 energy units are needed to send a message
            print(f"Robot {self.robot_id} is sending a message to Robot {other_robot.robot_id}: {message}")
            other_robot.receive_message(message)
            self.energy -= 10
        else:
            print(f"Robot {self.robot_id} does not have enough energy to send a message.")

    def receive_message(self, message):
        print(f"Robot {self.robot_id} received a message: {message}")
        self.messages_received.append(message)

class CommunicationModel(nn.Module):
    def __init__(self):
        super(CommunicationModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

def main():
    ranet = []

    # Creating robots with initial positions
    for i in range(5):
        ranet.append(Robot(i + 1, random.randint(0, 10), random.randint(0, 10)))

    communication_model = CommunicationModel()
    optimizer = optim.Adam(communication_model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()

    # Training the communication model
    train_communication = input("Do you want to train the communication model? (yes/no): ").lower() == 'yes'

    if train_communication:
        num_epochs = int(input("Enter the number of training epochs: "))
        for epoch in range(num_epochs):
            for robot in ranet:
                for other_robot in ranet:
                    if robot != other_robot:
                        # Simulate communication
                        input_data = torch.tensor([robot.x, robot.y, other_robot.x, other_robot.y], dtype=torch.float32).unsqueeze(0)
                        target = torch.tensor([1.0 if random.random() < 0.5 else 0.0], dtype=torch.float32).unsqueeze(0)

                        optimizer.zero_grad()
                        output = communication_model(input_data)
                        loss = criterion(output, target)
                        loss.backward()
                        optimizer.step()

                        # Send a message based on the model's prediction
                        if output.item() > 0.5:
                            robot.send_message(other_robot, f"Hello from Robot {robot.robot_id}!")

    # Get sender and receiver input from the user
    sender_id = int(input("Enter the ID of the sender robot: "))
    receiver_id = int(input("Enter the ID of the receiver robot: "))
    sender = next((robot for robot in ranet if robot.robot_id == sender_id), None)
    receiver = next((robot for robot in ranet if robot.robot_id == receiver_id), None)

    if sender is not None and receiver is not None:
        sender.send_message(receiver, f"Direct message from Robot {sender.robot_id} to Robot {receiver.robot_id}!")

    # Displaying final info
    for robot in ranet:
        print(f"Robot {robot.robot_id}: Position=({robot.x}, {robot.y}), Energy={robot.energy}")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    main()


In [ ]:
#updated energy basic
import torch
import torch.nn as nn
import torch.optim as optim
import random

# MAC Protocol Constants
PROTOCOLS = ["Protocol1", "Protocol2", "Protocol3", "Protocol4", "Protocol5"]

class Robot:
    def __init__(self, robot_id, x, y, energy=100, mac_protocol=None):
        self.robot_id = robot_id
        self.x = x
        self.y = y
        self.energy = energy
        self.mac_protocol = mac_protocol
        self.messages_received = []

    def move(self, new_x, new_y):
        distance = ((new_x - self.x) ** 2 + (new_y - self.y) ** 2) ** 0.5
        energy_consumed = distance * 2
        if self.energy >= energy_consumed:
            print(f"Robot {self.robot_id} is moving from ({self.x}, {self.y}) to ({new_x}, {new_y})")
            self.x = new_x
            self.y = new_y
            self.energy -= energy_consumed
            print(f"Energy consumed: {energy_consumed}, Remaining energy: {self.energy}")
        else:
            print(f"Robot {self.robot_id} does not have enough energy to move.")

    def send_message(self, other_robot, message):
        print(f"Robot {self.robot_id} is sending a message to Robot {other_robot.robot_id}: {message}")
        other_robot.receive_message(message)

    def receive_message(self, message):
        print(f"Robot {self.robot_id} received a message: {message}")
        self.messages_received.append(message)

class CommunicationModel(nn.Module):
    def __init__(self):
        super(CommunicationModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.fc(x)

def main():
    ranet = []

    for i in range(5):
        ranet.append(Robot(i + 1, random.randint(0, 10), random.randint(0, 10), mac_protocol=PROTOCOLS[i]))

    communication_model = CommunicationModel()
    optimizer = optim.Adam(communication_model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()

    while True:
        sender_id = int(input("Enter the ID of the sender robot (or enter -1 to exit): "))
        if sender_id == -1:
            break

        receiver_id = int(input("Enter the ID of the receiver robot: "))

        sender = next((r for r in ranet if r.robot_id == sender_id), None)
        receiver = next((r for r in ranet if r.robot_id == receiver_id), None)

        # Prompt for message input here
        message = input(f"Enter the message from Robot {sender_id} to Robot {receiver_id}: ")

        if sender is not None and receiver is not None:
            # Simulate communication
            input_data = torch.tensor([sender.x, sender.y, receiver.x, receiver.y], dtype=torch.float32).unsqueeze(0)
            target = torch.tensor([1.0 if random.random() < 0.5 else 0.0], dtype=torch.float32).unsqueeze(0)

            optimizer.zero_grad()
            output = communication_model(input_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            # Send a message based on the model's prediction
            if output.item() > 0.5:
                sender.send_message(receiver, message)
                sender.move(random.randint(0, 10), random.randint(0, 10))  # Update the energy after sending a message

        else:
            print("Invalid sender or receiver ID. Please try again.")

    # Displaying final info
    for robot in ranet:
        print(f"Robot {robot.robot_id}: Position=({robot.x}, {robot.y}), Energy={robot.energy}, MAC Protocol={robot.mac_protocol}")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    main()

In [ ]:
#location finding protocol
import torch
import torch.nn as nn
import torch.optim as optim
import random

# MAC Protocol Constants
PROTOCOLS = ["Protocol1", "Protocol2", "Protocol3", "Protocol4", "Protocol5"]

class Robot:
    def __init__(self, robot_id, x, y, energy=100, mac_protocol=None):
        self.robot_id = robot_id
        self.x = x
        self.y = y
        self.energy = energy
        self.mac_protocol = mac_protocol
        self.messages_received = []

    def move(self, new_x, new_y):
        distance = ((new_x - self.x) ** 2 + (new_y - self.y) ** 2) ** 0.5
        energy_consumed = distance * 2
        if self.energy >= energy_consumed:
            print(f"Robot {self.robot_id} is moving from ({self.x}, {self.y}) to ({new_x}, {new_y})")
            self.x = new_x
            self.y = new_y
            self.energy -= energy_consumed
            print(f"Energy consumed: {energy_consumed}, Remaining energy: {self.energy}")
        else:
            print(f"Robot {self.robot_id} does not have enough energy to move.")

    def send_message(self, other_robot, message):
        print(f"Robot {self.robot_id} is sending a message to Robot {other_robot.robot_id}: {message}")
        other_robot.receive_message(message)

    def receive_message(self, message):
        print(f"Robot {self.robot_id} received a message: {message}")
        self.messages_received.append(message)

def average_positions(robots):
    total_x = sum(robot.x for robot in robots)
    total_y = sum(robot.y for robot in robots)
    return total_x / len(robots), total_y / len(robots)

def main():
    ranet = []

    for i in range(5):
        ranet.append(Robot(i + 1, random.randint(0, 10), random.randint(0, 10), mac_protocol=PROTOCOLS[i]))

    communication_model = nn.Module()  # Dummy model, not used in this example

    while True:
        sender_id = int(input("Enter the ID of the sender robot (or enter -1 to exit): "))
        if sender_id == -1:
            break

        receiver_id = int(input("Enter the ID of the receiver robot: "))

        sender = next((r for r in ranet if r.robot_id == sender_id), None)
        receiver = next((r for r in ranet if r.robot_id == receiver_id), None)

        if sender is not None and receiver is not None:
            # Simulate communication
            message = input(f"Enter the message from Robot {sender_id} to Robot {receiver_id}: ")
            sender.send_message(receiver, message)

            # Update positions based on the location-finding protocol (averaging)
            average_x, average_y = average_positions(ranet)
            for robot in ranet:
                robot.x, robot.y = average_x, average_y

        else:
            print("Invalid sender or receiver ID. Please try again.")

    # Displaying final info
    for robot in ranet:
        print(f"Robot {robot.robot_id}: Position=({robot.x}, {robot.y}), Energy={robot.energy}, MAC Protocol={robot.mac_protocol}")
        print(f"Messages received by Robot {robot.robot_id}: {robot.messages_received}")

if __name__ == "__main__":
    main()
